In [1]:
import xgboost as xbg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import pickle
import joblib

%matplotlib inline
# comment 1
#comm 2

In [2]:
def load_cls(fname):
    return np.array(pickle.load(open(fname,"rb")))
# add path to .p file
path = "/scratch/pbanerj6/sml-dataset/"

X_train_cls = load_cls(path+"X_train_cls.p")
X_val_cls = load_cls(path+"X_val_cls.p")

In [3]:
y_train = load_cls(path+"y_train.p")
y_val = load_cls(path+"y_val.p")

In [40]:
#from xgboost import XGBClassifier
#model = XGBClassifier(n_jobs=-1,verbosity=2)

from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors=5,  algorithm='auto',n_jobs=-1)

In [40]:
model.fit(X_train_cls,y_train)

In [55]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

parameters = {
     'n_neighbors': [7,9,11]
 }

knn = KNeighborsClassifier()

grid = GridSearchCV(knn, parameters, cv = 5, n_jobs=-1, verbose=4)  # scoring = 'accuracy'

In [ ]:
model= grid.fit(X_train_cls,y_train)

Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 20 concurrent workers.


In [ ]:
print (model.best_score_)
print (model.best_params_)
print (model.best_estimator_)

print(model.best_params_)
for param_name in sorted(parameters.keys()):
    print("%s: %r" % (param_name, model.best_params_[param_name]))

In [42]:
from sklearn.metrics import accuracy_score

In [43]:
preds_val = model.predict(X_val_cls)

In [44]:
accuracy = accuracy_score(y_val, preds_val)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 56.03%


In [45]:
X_test_cls = load_cls(path+"X_test_cls.p")
y_test = load_cls(path+"y_test.p")

In [46]:
print(len(X_train_cls), len(X_test_cls), len(X_val_cls))

75000 15000 15000


In [48]:
preds_test = model.predict(X_test_cls)
accuracy = accuracy_score(y_test, preds_test)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

KeyboardInterrupt: 

In [49]:
X_test_cls[0].shape[0]

768

In [50]:
from scipy.stats import rankdata

def mrrs(out, labels):
#     print(out,labels)
    outputs = np.argmax(out,axis=1)
    mrr = 0.0 
    for label,ranks in zip(labels,out):
        ranks = rankdata(ranks*-1)
        rank = ranks[label]
#         print(rank,ranks)
        mrr+=1/rank
    return mrr/len(labels)

def mrrwrapper(qid2c,qid2indexmap,preds_prob):
    labels = []
    out = []
    for qid in qid2c.keys():
        scores = []
        for ix in qid2indexmap[qid]:
            if len(scores) < 6:
                scores.append(preds_prob[ix][1])
        if len(scores) < 6:
            continue
        out.append(scores)
        labels.append(int(qid2c[qid]))
    return mrrs(np.array(out),labels)

def load_ranking(fname):
    return pickle.load(open(path+"ranking_"+fname+".p","rb"))

In [51]:
preds_test_probs = model.predict_proba(X_test_cls)

In [52]:
qid2c,qid2indexmap = load_ranking("test")

In [53]:
print("MRR= %.4f" % (mrrwrapper(qid2c,qid2indexmap,preds_test_probs)))

# mrrwrapper(qid2c,qid2indexmap,preds_test_probs)

MRR= 0.4549
